# Benchmark for signal representation 

- Multifractal analysis (1ere approche)

- Discrete Fourier Transform (DFT) $\checkmark$
- Spectrogram
- Autoregression $\checkmark$
- Shannon encoding $\checkmark$
- Wavelets (en cours)

- Local symbolic features
- SAX representation
- Approximate entropy

ML

- Autoencoder

- RNN
- LSTM



## Imports

In [1]:
import numpy as np 
import matplotlib.pyplot as plt

import random
import itertools
# import pywt

# import pymultifracs.mfa as mfa
# from pymultifracs.utils import build_q_log
# from statsmodels.tsa.ar_model import AutoReg, ar_select_order


In [2]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler


In [3]:
# %pip install import_ipynb
# %pip install  --user git+https://github.com/neurospin/pymultifracs

In [4]:
import import_ipynb
#from transformations import DataTransform,TransformationRegistry, IdentityTransform, FourierTransform, LowFourierTransform, LowPsdTransform, WaveDecTransform, DwtTransform, CwtTransform, AutoRegTransform, ShannonEncodingTransform, WaveletLeadersTransform, CrossCorTransform, AutoCorTransform, MultiFracsTransform, AutoEncoderTransform  
from transformations import *

importing Jupyter notebook from transformations.ipynb


In [5]:
# Initialize the registry
registry = TransformationRegistry()

# Register transformations
registry.register('identity', IdentityTransform)
registry.register('mean', Mean)
registry.register('std', StandardDeviation)
registry.register('fourier', FourierTransform)
registry.register('low_fourier', LowFourierTransform)
registry.register('low_psd', LowPsdTransform)
registry.register('wavedec', WaveDecTransform)
registry.register('dwt', DwtTransform)
registry.register('cwt', CwtTransform)
registry.register('autoreg', AutoRegTransform)
registry.register('shannon_encoding', ShannonEncodingTransform)
registry.register('wavelet_leaders', WaveletLeadersTransform)
registry.register('multifracs', MultiFracsTransform)
registry.register('crosscor', CrossCorTransform)
registry.register('autocor', AutoCorTransform)
registry.register('autoencoder', AutoEncoderTransform)

## Data Load

Link here [ecgs_labels.npy](https://drive.google.com/file/d/1cbUKH9qGOeIZD6Mf73plMkyXpq56mwIu/view?usp=sharing)

In [6]:
DATASET = "ECG"
# DATASET = "HRV"
DATASET = "HRV_chall2002"
# DATASET = "SYNTH_DATA"

In [7]:
if DATASET == "ECG":
    ecgs_labels = np.load('ecgs_labels.npy')

    X, y = ecgs_labels[1:,:-1], ecgs_labels[1:,-1]

In [8]:
if DATASET ==  "HRV":
    X, y = np.load('hrv_signals.npy'), np.load('hrv_labels.npy')

In [9]:
if DATASET ==  "HRV_chall2002":
    hrvs_labels = np.load('hrv_data.npy')

    X, y = hrvs_labels[:,:-1], hrvs_labels[:,-1]

In [10]:
if DATASET ==  "SYNTH_DATA":
    data = np.load('synth_data.npy')

    X, y = data[:,:-1], data[:,-1]

In [11]:
np.isnan(X).sum()

0

In [12]:
n,p = X.shape

print(f'X.shape : {n,p}')

X.shape : (68, 71933)


In [13]:
j2max = min(12,int(np.log2(p) - 3))
j2max

12

In [14]:
p_ = 65000

In [15]:
data_transformer = DataTransform(registry,save_data=False)
for trans_names in registry.transformations.keys():
        trans_names_str = [str(name) for name in trans_names]
        trans_name_str = '+'.join(trans_names_str) if isinstance(trans_names, list) else trans_names
        kwargs = trans_names[1] if isinstance(trans_names, list) and len(trans_names) > 1 else {}
        trans_names = trans_names[0] if isinstance(trans_names, list) else trans_names
        # Apply transformation
        transformed_X = data_transformer.apply_transformation(np.random.randn((10*p_)).reshape((10,p_)), trans_names, **kwargs)
        
        print(f"Transformation: {trans_name_str}, Shape: {transformed_X.shape}")

Computing  identity ...

Transformation: identity, Shape: (10, 65000)
Transformation: mean, Shape: (10, 1)
Transformation: std, Shape: (10, 1)
Transformation: fourier, Shape: (10, 65000)
Transformation: low_fourier, Shape: (10, 1092)
Transformation: low_psd, Shape: (10, 1092)
Transformation: wavedec, Shape: (10, 4063)
Transformation: dwt, Shape: (10, 32500)
Transformation: cwt, Shape: (10, 10)
Transformation: autoreg, Shape: (10, 3)
Transformation: shannon_encoding, Shape: (10, 8)
Transformation: wavelet_leaders, Shape: (10, 2)
Transformation: multifracs, Shape: (10, 3)
Transformation: crosscor, Shape: (10, 10)
Transformation: autocor, Shape: (10, 26000)
Transformation: autoencoder, Shape: (10, 16)


In [16]:
# Initialize the data transformer
data_transformer = DataTransform(registry)

In [17]:
%%script false --no-raise-error

# Define the classifiers to be tested
classifiers = {
    'SVM': SVC(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier()
}

# Define the transformations to be tested
transformations = [
    # ['identity'],
    ['crosscor'],
    ['autocor', {'m':5000,'k':4}],
    ['fourier', {'new_dimension':40}],
    ['low_fourier'],
    ['low_psd'],
    ['cwt',{'pca_components' : 10}],
    ['wavedec'],
    ['autoreg', {'k': 3}],
    ['shannon_encoding'],
    ['wavelet_leaders'],
    ['multifracs'],
    ['multifracs', {'j1':1,'j2':12}],
    [['wavelet_leaders','shannon_encoding']],
    [['wavelet_leaders','multifracs']],
    [['fourier','multifracs',], {'new_dimension':40}],
    [['fourier','multifracs',], {'new_dimension':40}],
    [['fourier','multifracs','shannon_encoding'], {'new_dimension':40}],
    [['low_fourier','multifracs','autoreg'], {'k':3}],
    
]




# Function to evaluate a classifier using cross-validation
def evaluate_classifier_cv(classifier, X, y):
    scores = cross_val_score(classifier, X, y, cv=5)  # 5-fold cross-validation
    return np.mean(scores), np.std(scores)

# Loop over each transformation and each classifier
results = {}

for trans_names in transformations:
    # print()
    trans_names_str = [str(name) for name in trans_names]
    trans_name_str = '+'.join(trans_names_str) if isinstance(trans_names, list) else trans_names
    kwargs = trans_names[1] if isinstance(trans_names, list) and len(trans_names) > 1 else {}
    trans_names = trans_names[0] if isinstance(trans_names, list) else trans_names
    
    # Apply transformation
    transformed_X = data_transformer.apply_transformation(X, trans_names, **kwargs)
    print(f"Transformation: {trans_name_str}, Shape: {transformed_X.shape}" )
    # Standardize the data (important for some classifiers like SVM)
    scaler = StandardScaler()
    transformed_X = scaler.fit_transform(transformed_X)
    
    results[trans_name_str] = {}
    for clf_name, clf in classifiers.items():
        # Evaluate the classifier with cross-validation
        mean_accuracy, std_accuracy = evaluate_classifier_cv(clf, transformed_X, y)
        results[trans_name_str][clf_name] = (mean_accuracy, std_accuracy)
        print(f"Transformation: {trans_name_str}, Classifier: {clf_name}, Mean Accuracy: {mean_accuracy:.3f}, Std Dev: {std_accuracy:.3f}")

    print()
# Print the results
for trans_name, clf_results in results.items():
    print()
    for clf_name, (mean_accuracy, std_accuracy) in clf_results.items():
        print(f"Transformation: {trans_name}, Classifier: {clf_name}, Mean Accuracy: {mean_accuracy:.3f}, Std Dev: {std_accuracy:.3f}")
        

Couldn't find program: 'false'


In [18]:
%%script false --no-raise-error
import random
import itertools
import json

# Define the classifiers to be tested
classifiers = {
    'SVM': SVC(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier()
}

# Initialize the data transformer
data_transformer = DataTransform(registry)

# Define the transformations to be tested
transformation_names = ['crosscor','low_psd','low_fourier',['autoreg',{'k':3}],['autoreg',{'k':5}], #'multifracs',
                        ['multifracs', {'j1':1,'j2':j2max}],
                        'shannon_encoding',['autoencoder',{'fourier_transform':True}]] #list(registry.transformations.keys())




# Function to evaluate a classifier using cross-validation
def evaluate_classifier_cv(classifier, X, y):
    scores = cross_val_score(classifier, X, y, cv=5)  # 5-fold cross-validation
    return np.mean(scores), np.std(scores)

# Function to randomly combine transformations
def random_combination_transformations(transformation_list, n_combinations=10):
    # all_combinations = []
    # for r in range(1, len(transformation_names) + 1):
    #     combinations = list(itertools.combinations(transformation_names, r))
    #     all_combinations.extend(combinations)
    
    # return random.sample(all_combinations, min(n_combinations, len(all_combinations)))
    nb_transformation = len(transformation_list)
    combined_transformations = []
    for _ in range(n_combinations):
        nb_trans = np.random.randint(1,5)

        already_drawn = []
        transs = []
        for _ in range(nb_trans):
            while True:
                random_ind = np.random.randint(nb_transformation)
                if random_ind not in already_drawn:
                    already_drawn.append(random_ind)
                    break
            trans = transformation_list[random_ind]
            transs.append(trans)
            
        combined_transformations.append(transs)
    return combined_transformations
# Generate random combinations of transformations
random_transformations = random_combination_transformations(transformation_names, n_combinations=5)
print(random_transformations)
# # Example input data
# X = np.random.randn(100, 10)  # Example input data
# y = np.random.randint(0, 2, 100)  # Example labels

# Dictionary to store results
results = {}

i = 0
# Loop over each random combination of transformations and each classifier
for trans in random_transformations:
    print()
    trans_name_str = DataTransform.get_full_trans_kwargs_str(trans)
    
        
    transformed_X = data_transformer.apply_transformation(X, trans)

    # trans_name_str = '+'.join(trans_comb)
    # transformed_X = data_transformer.apply_transformation(X, trans_comb)
    
    # Standardize the data (important for some classifiers like SVM)
    scaler = StandardScaler()
    transformed_X = scaler.fit_transform(transformed_X)
    
    results[trans_name_str] = {}
    i += 1
    print(f"Transformations n°{i}: {trans_name_str} {transformed_X.shape}")
    for clf_name, clf in classifiers.items():
        # Evaluate the classifier with cross-validation
        mean_accuracy, std_accuracy = evaluate_classifier_cv(clf, transformed_X, y)
        results[trans_name_str][clf_name] = {'mean_accuracy': mean_accuracy, 'std_accuracy': std_accuracy}
        print(f"Classifier: {clf_name}, Mean Accuracy: {mean_accuracy:.3f}, Std Dev: {std_accuracy:.3f}")

# Save results to a JSON file
with open('transformation_results0.json', 'w') as f:
    json.dump(results, f, indent=4)


Couldn't find program: 'false'


In [19]:
# # Save results to a JSON file
# with open('transformation_results_hrv_chall2002.json', 'w') as f:
#     json.dump(results, f, indent=4)

In [20]:
# import json
# ##
# # Load the results from the JSON file
# with open('transformation_results_hrv_chall2002.json', 'r') as f:
#     results = json.load(f)

# # Find the best transformation and classifier
# best_score = 0
# best_transformation = None
# best_classifier = None

# for trans_comb, clf_results in results.items():
#     for clf_name, scores in clf_results.items():
#         if scores['mean_accuracy'] > best_score:
#             best_score = scores['mean_accuracy']
#             best_transformation = trans_comb
#             best_classifier = clf_name

# print(f"Best Transformation: {best_transformation}")
# print(f"Best Classifier: {best_classifier}")
# print(f"Best Mean Accuracy: {best_score}")


In [21]:
def random_combination_transformations(transformation_names, k = 2):
    all_combinations = []
    for r in range(1, k + 1):
        combinations = list(itertools.combinations(transformation_names, r))
        all_combinations.extend(combinations)
    
    return [list(x) for x in all_combinations] 


In [22]:
import pandas as pd
from sklearn.model_selection import  KFold


# Define the classifiers
classifiers = {
    'SVM': SVC(kernel='linear'),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier()
}

# # Define the data transformations
# transformations = [
#     'identity',
#     ['fourier', {'new_dimension': 100}],
#     ['wavedec', {'level': 4, 'wavelet': 'db1'}],
#     ['autoreg', {'k': 1}]
# ]
# Define the transformations to be tested
transformations = ['crosscor','mean','std',
                    #'low_psd','low_fourier',
                    #['autoreg',{'k':1}], ['autoreg',{'k':2}],['autoreg',{'k':3}],
                    ['autoreg',{'k':5}], #'multifracs',
                    ['multifracs', {'j1':1,'j2':j2max}],
                    'shannon_encoding',['autoencoder',{'fourier_transform':True}]]

if DATASET == "ECG":
    transformations.extend(["low_psd","low_fourier"])

t = random_combination_transformations(transformations,k=4)
transformations = t
print(transformations)
def evaluate_transformations(X, y, transformations, classifiers):
    dt = DataTransform(registry)
    results = []

    for trans in transformations:
        trans_name_str = DataTransform.get_full_trans_kwargs_str(trans)
        trans_X = dt.apply_transformation(X, trans)

        result = {'Transformation (shape)': trans_name_str}
        result["shape"] = trans_X.shape[-1]
        result['nb_trans'] = len(trans)
        for clf_name, clf in classifiers.items():
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            scores = cross_val_score(clf, trans_X, y, cv=kf, scoring='accuracy')
            result[f'{clf_name} accuracy'] = np.mean(scores)
            result[f'{clf_name} std'] = np.std(scores)
            print(f"Transformation: {trans_name_str}, Classifier: {clf_name}, "
                  f"Mean Accuracy: {np.mean(scores):.3f}, Std Accuracy: {np.std(scores):.3f}")
        
        results.append(result)

    return pd.DataFrame(results)


results_df = evaluate_transformations(X, y, transformations, classifiers)
print(results_df)

# Save results to a CSV file
results_df.to_csv(f'results/{DATASET}_transformation_results_.csv', index=False)


[['crosscor'], ['mean'], ['std'], [['autoreg', {'k': 5}]], [['multifracs', {'j1': 1, 'j2': 12}]], ['shannon_encoding'], [['autoencoder', {'fourier_transform': True}]], ['crosscor', 'mean'], ['crosscor', 'std'], ['crosscor', ['autoreg', {'k': 5}]], ['crosscor', ['multifracs', {'j1': 1, 'j2': 12}]], ['crosscor', 'shannon_encoding'], ['crosscor', ['autoencoder', {'fourier_transform': True}]], ['mean', 'std'], ['mean', ['autoreg', {'k': 5}]], ['mean', ['multifracs', {'j1': 1, 'j2': 12}]], ['mean', 'shannon_encoding'], ['mean', ['autoencoder', {'fourier_transform': True}]], ['std', ['autoreg', {'k': 5}]], ['std', ['multifracs', {'j1': 1, 'j2': 12}]], ['std', 'shannon_encoding'], ['std', ['autoencoder', {'fourier_transform': True}]], [['autoreg', {'k': 5}], ['multifracs', {'j1': 1, 'j2': 12}]], [['autoreg', {'k': 5}], 'shannon_encoding'], [['autoreg', {'k': 5}], ['autoencoder', {'fourier_transform': True}]], [['multifracs', {'j1': 1, 'j2': 12}], 'shannon_encoding'], [['multifracs', {'j1': 1,

c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\utils.py:87: RuntimeWarning: divide by zero encountered in power
  return np.power(array, exponent)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\utils.py:76: RuntimeWarning: divide by zero encountered in reciprocal
  return array ** exponent
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\cumulants.py:265: RuntimeWarning: divide by zero encountered in log
  log_T_X_j = np.log(T_X_j)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:117: RuntimeWarning: invalid value encountered in divide
  R_j = temp / Z
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:118: RuntimeWarning: divide by zero encountered in log2
  V[:, ind_j, :] = fixednansum(R_j * np.log2(mrq_values_j), axis=1)
c:\users\ant\documents\info2\bits2beat-tests\pymultifracs\pymultifracs\mfspectrum.py:118: RuntimeWarning: invalid value encoun

Transformation: multifracs_j1=1_j2=12, Classifier: SVM, Mean Accuracy: 0.733, Std Accuracy: 0.069
Transformation: multifracs_j1=1_j2=12, Classifier: DecisionTree, Mean Accuracy: 0.602, Std Accuracy: 0.118
Transformation: multifracs_j1=1_j2=12, Classifier: RandomForest, Mean Accuracy: 0.688, Std Accuracy: 0.097
Transformation: shannon_encoding, Classifier: SVM, Mean Accuracy: 0.715, Std Accuracy: 0.147
Transformation: shannon_encoding, Classifier: DecisionTree, Mean Accuracy: 0.708, Std Accuracy: 0.057
Transformation: shannon_encoding, Classifier: RandomForest, Mean Accuracy: 0.748, Std Accuracy: 0.064
Transformation: autoencoder_fourier_transform=True, Classifier: SVM, Mean Accuracy: 0.733, Std Accuracy: 0.069
Transformation: autoencoder_fourier_transform=True, Classifier: DecisionTree, Mean Accuracy: 0.544, Std Accuracy: 0.094
Transformation: autoencoder_fourier_transform=True, Classifier: RandomForest, Mean Accuracy: 0.573, Std Accuracy: 0.074
Transformation: crosscor_mean, Classifie

In [23]:
results_df

,Transformation (shape),shape,nb_trans,SVM accuracy,SVM std,DecisionTree accuracy,DecisionTree std,RandomForest accuracy,RandomForest std
0,crosscor,10,1,0.732967,0.069030,0.587912,0.036776,0.630769,0.056612
1,mean,1,1,0.732967,0.069030,0.630769,0.106622,0.630769,0.106622
2,std,1,1,0.732967,0.069030,0.573626,0.051379,0.573626,0.051379
3,autoreg_k=5,5,1,0.747253,0.084193,0.718681,0.078200,0.823077,0.037812
4,multifracs_j1=1_j2=12,3,1,0.732967,0.069030,0.602198,0.118079,0.687912,0.097388
...,...,...,...,...,...,...,...,...,...
93,std_autoreg_k=5_multifracs_j1=1_j2=12_shannon_...,17,4,0.791209,0.122516,0.718681,0.078200,0.806593,0.066518
94,std_autoreg_k=5_multifracs_j1=1_j2=12_autoenco...,25,4,0.747253,0.095548,0.762637,0.063070,0.762637,0.091572
95,std_autoreg_k=5_shannon_encoding_autoencoder_f...,30,4,0.792308,0.061617,0.748352,0.045760,0.821978,0.079136
96,std_multifracs_j1=1_j2=12_shannon_encoding_aut...,28,4,0.806593,0.066518,0.703297,0.131685,0.747253,0.095548
